In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(r'C:\Users\berka\Downloads\turkish_movie_sentiment_dataset.csv', sep=",", encoding="utf-8")

In [13]:
df.comment = df.comment.replace('\n','', regex=True)

In [14]:
df.comment = df.comment.replace('\r','', regex=True)

In [17]:
df.comment.describe()

count                    83227
unique                   82451
top                           
freq                       300
Name: comment, dtype: object

In [23]:
df.dtypes

comment      object
film_name    object
point        object
dtype: object

In [43]:
for i in range(len(df.point)):
    df.point[i] = df.point[i][0]

In [51]:
df['comment'] = df['comment'].astype('string') 

In [48]:
df['point'] = df['point'].astype('string') 

In [52]:
df.dtypes

comment      string
film_name    object
point        string
dtype: object

In [56]:
df.drop(columns=["film_name"], inplace=True)

In [71]:
for i in range(len(df.comment)):
    df.comment[i].encode('utf-8')

In [19]:
embedding_layer = tf.keras.layers.Embedding(1000, 6)

In [20]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split

In [76]:
df

,comment,point
0,Jean Reno denince zaten ...,5
1,Ekşın falan izlemek isti...,5
2,Bu yapım hakkında öyle ç...,5
3,finali yeter... (sting -...,5
4,Jean Reno..bu adam kusur...,5
...,...,...
83222,"Böyle bi kadrodan, bçyle...",2
83223,yani bu kaar ii oyuncalr...,2
83224,bugün dvd'sini alıp izle...,1
83225,Klasik korku ve gerilim....,4


In [72]:
tf.convert_to_tensor(df)

<tf.Tensor: shape=(83227, 2), dtype=string, numpy=
array([[b'                      Jean Reno denince zaten leon filmi gelir akla izlemeyen kalmam\xc4\xb1\xc5\x9ft\xc4\xb1r ama kald\xc4\xb1ysada ee ne duruyorsun hemen izle :)                    ',
        b'5'],
       [b'                      Ek\xc5\x9f\xc4\xb1n falan izlemek istiyorsan\xc4\xb1z e\xc4\x9fer bunu izlemeyiin dostlar\xc4\xb1m keza ilk sahne hari\xc3\xa7 ek\xc5\x9f\xc4\xb1n filmde yerini gittik\xc3\xa7e duygusall\xc4\xb1\xc4\x9fa b\xc4\xb1rak\xc4\xb1r.. Mathildan\xc4\xb1n Leonun evine geldi\xc4\x9finde a\xc4\x9flad\xc4\xb1\xc4\x9f\xc4\xb1 sahnede ben de a\xc4\x9flam\xc4\xb1\xc5\x9f olabilirim.. Tamam olabilirim de\xc4\x9fil a\xc4\x9flad\xc4\xb1m; ama sen de izle a\xc4\x9flars\xc4\xb1n dostooom. Etkileyici bir film, s\xc4\xb1km\xc4\xb1yor hi\xc3\xa7bir salise boyunca sizi." Ben art\xc4\xb1k b\xc3\xbcy\xc3\xbcd\xc3\xbcm Leon, ya\xc5\x9flan\xc4\xb1yorum. "" Hayat hep b\xc3\xb6yle zor mu, yoksa sadece \xc3\xa7ocukken mi? "    

In [60]:
train_ds = df.sample(n=60000, random_state=1)

In [54]:
# X_train, X_test, y_train, y_test = train_test_split(df.comment, df.point, test_size=0.25, random_state=42)

In [74]:
tf.convert_to_tensor(train_ds)

<tf.Tensor: shape=(60000, 2), dtype=string, numpy=
array([[b'                      ben bu kadar g\xc3\xbczel bir film ne zamand\xc4\xb1r seyretmiyodum filmde a\xc4\x9flad\xc4\xb1m ya :D ger\xc3\xa7ekten bence s\xc3\xbcper film                    ',
        b'5'],
       [b'                      En yak\xc4\xb1n dostumun "E\xc5\x9f\xc5\x9fek kadar adam oldunuz hala bu filmleri mi seyrediosunuz" \xc5\x9feklindeki yorumuna sert bir dille cevap verdi\xc4\x9fim film:)Klasik amerikan gen\xc3\xa7li\xc4\x9fi filmi de olsa ya\xc5\x9fataca\xc4\x9f\xc4\xb1 e\xc4\x9flenceli dakikalar i\xc3\xa7in bile izlenir. 10/7                    ',
        b'4'],
       [b"                      son zamanlarda seyretti\xc4\x9fim en k\xc3\xb6t\xc3\xbc film ata'n\xc4\xb1n ismi i\xc3\xa7in gittim filme ancak sonu\xc3\xa7 h\xc3\xbcsran\xc5\x9febnem han\xc4\xb1m'da kurtaramam\xc4\xb1\xc5\x9f filmi                    ",
        b'0'],
       ...,
       [b'                      Sava\xc5\x9f filmlerini severim. Atmosfe

In [75]:
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

AttributeError: 'DataFrame' object has no attribute 'map'